# Imports

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer
from transformers import BertModel
from transformers import TFBertForSequenceClassification
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

In [2]:
tf.keras.backend.set_floatx('float16')

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Load data

In [4]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

# Define hyperparameters

In [13]:
tokenizer_max_length = 64
base_batch_size = 32
lr = 2e-5
num_epoch=10

# Tokenize

In [14]:
# load bert large model from transformers
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)
# encode train text and set padding length to 84.
train_encodings = tokenizer(list(train_df['clean_joined_text']), truncation=True, padding=True, max_length=tokenizer_max_length)

# Prepare data

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),
                                                    list(train_df['target'])))

# Instantiate model

In [8]:
model = TFBertForSequenceClassification.from_pretrained('bert-large-uncased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [15]:
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)

In [16]:
train_dataset.shuffle(len(train_dataset)).batch(base_batch_size)

<BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 42), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 42), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 42), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [18]:
model.fit(train_dataset.shuffle(len(train_dataset)).batch(base_batch_size),
          epochs=num_epoch,
          batch_size=base_batch_size)

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'tf_bert_for_sequence_classification/bert/encoder/layer_._16/attention/output/LayerNorm/moments/Cast' defined at (most recent call last):
    File "c:\Users\Rnowa\anaconda3\envs\SDA_DASCI\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Rnowa\anaconda3\envs\SDA_DASCI\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Rnowa\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Rnowa\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start()
    File "C:\Users\Rnowa\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\Rnowa\AppData\Roaming\Python\Python310\site-packages\tornado\platform

In [21]:
preds_bert = pd.read_csv('preds_df_bert.csv')


https://www.kaggle.com/code/afi1289/nlp-with-disaster-tweets-cleaning-tf-idf-and-bert/notebook?scriptVersionId=111089096
https://www.kaggle.com/code/tanknee/nlp-getting-started-with-bert-large/notebook?scriptVersionId=109267040